# Download NMR Dart survey data

**Author:** Logan Schmidt

**Created:** 2019-06-07

**Last Updated:** 2019-06-09

**Purpose:**

This script was created as part of the effort to produce the **Schmidt, Rempe 2019 Water Content** manuscript. This script accesses the NMR Dart survey data saved on the **DRempe Drive** on the JSG server and imports, processes, and saves the data as a single master csv file.

In [1]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import numpy as np
import os
import re
from time import gmtime, strftime

In [2]:
# select processed NMR folder to get directory
root = tk.Tk()
importDir = filedialog.askdirectory(title = 'Select Processed NMR Folder')
root.mainloop()
if len(importDir) > 0:
    print('You chose: %s' % importDir)

You chose: S:/DRempe Drive/Logan Schmidt/NMR/DATA/ERCZO/Processed_All/Processed_Survey


In [5]:
# get sitename for processed NMR folder
site = re.findall('/DATA/(\w*)', importDir)[0]
# get pathname for processed NMR folder
rootDir = os.path.abspath(importDir)
# create empty dataframe on which to append individual dataframes to create master dataframe
df2 = pd.DataFrame()

# walk through folders in NMR processed folder and import 1Dvector data files into a master dataframe
for (dirName, dirs, files) in os.walk(rootDir):
   for fileName in files:
       if fileName.startswith('well') & fileName.endswith('_1Dvectors_uniform.txt'):
            # get the name of the 1Dvectors file
            fileCrit = os.path.join(dirName,fileName)            
            # convert the *_1Dvectors_uniform.txt file into a dataframe
            df1 = pd.read_csv(fileCrit, sep = '\s+')

            # disect fileName to get wellID and year-month-day
            exps0  = re.findall('well_(\w*)', fileCrit)[1]
            exps1  = re.findall('(\w*?)_', exps0)
            wellID = exps1[0]
            year   = exps1[1]
            month  = exps1[2]
            day    = exps1[3]
            if day == '1Dvectors':
                day = '01'
            datetime = pd.to_datetime(year + month + day)

            # add wellID and date vectors
            df1.insert(0, 'wellID', wellID, True)
            df1.insert(1, 'date', datetime, True)
            
            df2 = df2.append(df1)

In [6]:
# get current datetime to append to filename
datetime = strftime("%Y-%m-%d-%H%M-UTC", gmtime())

# Save *working file*
df2.to_csv('Data/NMRDartSurveyRaw_' + site + '.csv', 
           index = False)
# Save time-stamped *back-up file*
df2.to_csv('Data/Old/' + datetime + ' NMRDartSurveyRaw_' + site + '.csv', 
           index = False)

# Read it in and check
pd.read_csv('Data/NMRDartSurveyRaw_' + site + '.csv',
            sep = ',').head()

,wellID,date,depth,totalf,clayf,capf,freef,mlT2,Ksdr,Ktc,Ksoe,Tsdr,Ttc,Tsoe,soe,noise
0,1,2017-05-30,6.50,0.060924,0.012071,0.034541,0.014312,0.015640,0.132630,1.100000e-09,0.258125,0.528887,2.640000e-08,0.842936,0.000103,3.349139
1,1,2017-05-30,6.25,0.061411,0.003627,0.031975,0.025809,0.025723,0.361636,6.200000e-09,0.348897,0.495730,2.620000e-08,0.778404,0.000168,3.247500
2,1,2017-05-30,6.00,0.053488,0.025785,0.007505,0.020198,0.009300,0.041176,2.500000e-09,0.102793,0.405321,2.460000e-08,0.691180,0.000045,3.188238
3,1,2017-05-30,5.75,0.063552,0.028773,0.018960,0.015819,0.007621,0.032849,1.500000e-09,0.167675,0.395027,2.400000e-08,0.665482,0.000076,3.286134
4,1,2017-05-30,5.50,0.114223,0.060400,0.031012,0.022811,0.006735,0.046106,8.800000e-09,0.346079,0.386814,2.360000e-08,0.623563,0.000139,3.227186
